# Smartphone Spectrum Technology Analysis

## Description
This Jupyter Notebook processes smartphone data to analyze cellular technology (4G, 4.5G, 5G). It combines three logical steps:
- **Step 1: LTE-A Identification: identifying 4.5G in the data**
- **Step 2: IDC Data Integration**
- **Step 3: Spectrum Technology Assignment**

The code does the foolowing tasks:
1) Filter the distinction between 4g and 4.5 g. <br>
2) Extract the generation data from IDC file. <br>
3) Choose the max possiblev available spectrum tech in the smartphone. This is because all 5g phones have 4g, 4.5g capability. so we choose the max available between the two.


### INPUT:
- complete data( exchange rates IDC, Phonearena(Feb 9,2024).csv
- phone data from phonearena.csv
- H_Data_smartphone.csv

### Output:
- Final file with accurate `Spec_tech` assignments based on LTE-A and generation info.- FINAL DATA_1.csv

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

try:
    phone_data = pd.read_csv("/content/drive/MyDrive/OUTPUT DATA(FROM IDC)/complete data( exchange rates IDC, Phonearena(Feb 9,2024).csv")
    phonearena_ref = pd.read_csv("/content/drive/MyDrive/OUTPUT DATA(FROM IDC)/COMPLETE DATA FILEs/phone data from phonearena.csv")
    idc_data = pd.read_csv("/content/drive/MyDrive/IDC data/H_Data_smartphone.csv")
except FileNotFoundError as e:
    print(f"Error loading files: {e}")
    raise

In [ ]:
# Rename columns
phone_data = phone_data.rename(columns={
    "Model Name": "model_name",
    "\n\nCellular\n\n": "cellular",
    "Data Speed:": "data_speed",
    "Differences from the main variant:": "variant_differences",
    "Quarter": "quarter",
    "Brand": "brand",
    "Screen Size": "screen_size",
    "Technology:": "technology"
})

In [ ]:
# Combine columns
cellular_cols = ['cellular', 'data_speed', 'variant_differences']
phone_data['cellular_combined'] = phone_data[cellular_cols].apply(lambda x: ' '.join(x.dropna()), axis=1)

# Flag LTE-A support
phone_data['cellular_lte_a'] = phone_data['cellular_combined'].where(
    phone_data['cellular_combined'].str.contains("LTE-A", na=False), "Not LTE-A")
phone_data.loc[phone_data['cellular_lte_a'].str.contains("LTE-A", na=False), 'cellular_lte_a'] = '4.5'

In [ ]:
# Clean IDC data
idc_data['Brand'] = idc_data['Brand'].replace('LG Electronics', 'LG')
idc_data['Model Name'] = (idc_data['Brand'] + ' ' + idc_data['Model Name']).str.upper()
idc_data['Model Name'] = idc_data['Model Name'].str.replace(' ', '').str.replace("'", '')
idc_data = idc_data.rename(columns={
    'Model Name': 'model_name',
    'Brand': 'brand',
    'Screen Size': 'screen_size',
    'Storage (GB)': 'storage_gb',
    'RAM (GB)': 'ram_gb',
    'Generation': 'generation',
    'Quarter': 'quarter'
})

In [ ]:
# Merge
merged_data = pd.merge(
    phone_data,
    idc_data[['model_name', 'brand', 'screen_size', 'storage_gb', 'ram_gb', 'generation', 'quarter']],
    on=['quarter', 'brand', 'screen_size', 'model_name'],
    how='left'
)
merged_data.drop(columns=[col for col in merged_data.columns if 'Unnamed' in col], inplace=True)

In [ ]:
# Assign Spec_tech
merged_data['generation_1'] = merged_data['generation'].str.replace('G', '')

def get_max_tech(row):
    try:
        val1 = float(row['cellular_lte_a']) if row['cellular_lte_a'] != 'Not LTE-A' else 0
        val2 = float(row['generation_1']) if pd.notnull(row['generation_1']) else 0
        return str(max(val1, val2))
    except:
        return row['generation_1']

merged_data['spec_tech'] = merged_data.apply(get_max_tech, axis=1)

In [ ]:
# Clean up and save
merged_data = merged_data[merged_data['model_name'] != 'SAMSUNGGALAXYSFASCINATE']
merged_data.drop(columns=['generation_1'], inplace=True, errors='ignore')
merged_data.to_csv("/content/drive/MyDrive/OUTPUT DATA(FROM IDC)/COMPLETE DATA FILEs/FINAL DATA_1.csv", index=False)